In [ ]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 " "(KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
os.environ["GROQ_API_KEY"] = groq_api


In [2]:
import os
from langchain_community.document_loaders import TextLoader, WebBaseLoader, PDFMinerLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.vectorstores import Chroma
from groq import Groq
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.embeddings import HuggingFaceEmbeddings
import requests
import re
import gradio as gr

In [3]:
url = "https://www.parliament.go.ke/sites/default/files/2017-05/The_Constitution_of_Kenya_2010.pdf"
r = requests.get(url)
with open("constitution.pdf", "wb") as f:
    f.write(r.content)
    
loader = PDFMinerLoader("constitution.pdf")
docs_list = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500,
    chunk_overlap=500,
    separators=["ARTICLE", "CHAPTER", "\n\n"])

splits = text_splitter.split_documents(docs_list)

In [5]:
print(len(splits))

107


In [6]:
print(splits[0])
print(splits[1])

page_content='LAWS OF KENYA

THE CONSTITUTION OF KENYA, 2010

Published by the National Council for Law Reporting
with the Authority of the Attorney-General

www.kenyalaw.org
Constitution of Kenya, 2010

THE CONSTITUTION OF KENYA, 2010

ARRANGEMENT OF' metadata={'producer': 'LibreOffice 3.6', 'creator': 'Writer', 'creationdate': '2013-02-28T17:50:04+03:00', 'title': 'Rev. 2010]', 'author': 'Andare Jeffrey', 'total_pages': 193, 'source': 'constitution.pdf'}
page_content='ARTICLES

PREAMBLE

CHAPTER ONE—SOVEREIGNTY OF THE PEOPLE AND 

SUPREMACY OF THIS CONSTITUTION

1—Sovereignty of the people.
2—Supremacy of this Constitution.
3—Defence of this Constitution.

CHAPTER TWO—THE REPUBLIC

4—Declaration of the Republic.
5—Territory of Kenya.
6—Devolution and access to services.
7—National, official and other languages.
8—State and religion.
9—National symbols and national days.
10—National values and principles of governance.
11—Culture.

CHAPTER THREE—CITIZENSHIP

12—Entitlements of citize

In [7]:
#Embedding the documents and storing  them in a vector store
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(
    persist_directory="constitution_index",
    embedding_function=embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

C:\Users\Admin\AppData\Local\Temp\ipykernel_16076\1653097174.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\Admin\AppData\Local\Temp\ipykernel_16076\1653097174.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [8]:
llm = ChatGroq(
        model="llama-3.1-8b-instant",  
         max_tokens=500)

In [9]:
prompt = PromptTemplate(
    template="""You are an assistant for question-answering Kenyan constitution tasks.
Use the following documents to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Documents: {documents}
Answer:
""",
input_variables=["question", "documents"]
)

In [10]:
rag_chain = prompt | llm | StrOutputParser()

In [11]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain

    def run(self, question):
        documents = self.retriever.invoke(question)
        doc_texts = "\n".join([doc.page_content for doc in documents])
        
        # Get the answer from the LLM
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [12]:
rag_app = RAGApplication(retriever, rag_chain)

question = "What is the duty of the senate?"
answer = rag_app.run(question)

print("Answer:", answer)

Answer: According to Article 98 of the Kenyan Constitution, the Senate has the following duties:

- To represent the interests of the counties and their governments
- To represent the interests of the people of Kenya in the National Assembly, and
- To consider and approve Bills related to counties and national government revenue allocations

However, it's worth noting that Article 98 of the Constitution also gives the Senate some additional duties including, but not limited to:
- To protect the rights and interests of the people of Kenya
- To promote the economic and social development of the counties and the people of Kenya
- To oversee the national government's expenditure and revenue collection
- To consider and approve presidential appointments to certain positions.
- To have a right of initiative, that is, to introduce Bills in the Senate.

These duties are outlined in the Constitution to ensure that the Senate plays a significant role in the governance of Kenya.


In [13]:
def ask(question):
    answer = rag_app.run(question)
    print("Answer:", answer)

In [14]:
ask("Under what circumstances can a Kenyan citizen lose or have their citizenship revoked according to the Constitution of Kenya, 2010??")

Answer: According to Article 14 of the Constitution of Kenya, 2010, a Kenyan citizen can lose or have their citizenship revoked under the following circumstances:

1. Voluntary renunciation: A Kenyan citizen can renounce their citizenship voluntarily as provided for in Article 14(1).
2. By operation of the law: A Kenyan citizen can lose their citizenship by operation of the law, including by making a false declaration or statement with the intention of obtaining citizenship (Article 14(2)).
3. By order of the High Court: A Kenyan citizen can have their citizenship revoked by an order of the High Court on grounds of dual citizenship, or where the person has been convicted of a serious crime and the High Court so orders (Article 14(3) and (4)).

It's worth noting that the Constitution of Kenya, 2010, emphasizes that a Kenyan citizen cannot lose their citizenship except as provided for in the Constitution.


In [15]:
def gradio_interface(query: str):
    answer = rag_app.run(query)
    return (answer
    )

# Build the Gradio app
gui = gr.Interface(
    fn=gradio_interface,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Markdown(),
    title="Kenyan Constitution RAG",
    description="Provide a query and receive a response based on the Kenyan Constitution",
)

# Launch the app
if __name__ == "__main__":
    gui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
